In [1]:
import numpy as np
import matplotlib.pyplot as plt 
# import pyspark
import torch
plt.rcParams['figure.figsize'] = 10, 10
from utilities import *
import math
from sklearn.model_selection import train_test_split

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

In [3]:
class dataset(Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def __getitem__(self,ind):
        return self.X[ind],self.y[ind]
    def __len__(self):
        return len(self.X)

In [4]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

In [5]:
# Q1: What is the distribution of each of these 
def process_inhval(*args):
    for val in args: 
        if val == 'NA':
            yield 0
        else:
            yield float(val)

In [6]:
readfilename = 'PI_DataSet_6_19_all.txt'
r = open(readfilename)
header = process_line(next(r))
inh_start = header.index('FPV') 
inh_end = header.index('DRV')+1
vec_start = header.index('count_vec_start')
vec_end = len(header)

In [7]:
vals = []
vecs = []
for line in r:
    line = process_line(line)
    token = list(process_inhval(*line[inh_start:inh_end]))
    vals.append(token)
    vecs.append(read_vector(line,vec_start,vec_end))
r.close()

In [8]:
vals = np.array(vals)

In [9]:
vecs = np.array(vecs)
max_ = vecs.max()
vecs = vecs / max_ # this is not sparse

In [10]:
X_train, X_test, y_train, y_test = train_test_split(vecs,vals,test_size = 0.33)

In [11]:
X_train.shape

(277386, 210)

In [12]:
ds = dataset(X_train,y_train)

In [13]:
batchsize = 100000
batches = int(math.ceil(len(X_train)/batchsize))
print(batches)
epochs = 100

3


In [14]:
dataloader = DataLoader(ds, batch_size = batchsize)
model = nn.Sequential(torch.nn.Linear(210, 8))
loss_fn = RMSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2,weight_decay=0.001)

In [15]:
losses = [] 

In [16]:
for epoch in range(epochs):
    for i,b in enumerate(dataloader):
        x,y = b
        y_pred = model(x.float())
        loss = loss_fn(y_pred,y.float())
        losses.append(float(loss.data))
        model.zero_grad()
        loss.backward()
        optimizer.step()
        if i%10 == 0:
            print("epoch:",epoch,"batch:",i,"loss:",loss.data)

epoch: 0 batch: 0 loss: tensor(30.6189)
epoch: 1 batch: 0 loss: tensor(30.4529)
epoch: 2 batch: 0 loss: tensor(30.2890)
epoch: 3 batch: 0 loss: tensor(30.1273)
epoch: 4 batch: 0 loss: tensor(29.9677)
epoch: 5 batch: 0 loss: tensor(29.8102)
epoch: 6 batch: 0 loss: tensor(29.6549)
epoch: 7 batch: 0 loss: tensor(29.5017)
epoch: 8 batch: 0 loss: tensor(29.3507)
epoch: 9 batch: 0 loss: tensor(29.2018)
epoch: 10 batch: 0 loss: tensor(29.0550)
epoch: 11 batch: 0 loss: tensor(28.9104)
epoch: 12 batch: 0 loss: tensor(28.7680)
epoch: 13 batch: 0 loss: tensor(28.6277)
epoch: 14 batch: 0 loss: tensor(28.4895)
epoch: 15 batch: 0 loss: tensor(28.3535)
epoch: 16 batch: 0 loss: tensor(28.2196)
epoch: 17 batch: 0 loss: tensor(28.0878)
epoch: 18 batch: 0 loss: tensor(27.9581)
epoch: 19 batch: 0 loss: tensor(27.8305)
epoch: 20 batch: 0 loss: tensor(27.7051)
epoch: 21 batch: 0 loss: tensor(27.5817)
epoch: 22 batch: 0 loss: tensor(27.4604)
epoch: 23 batch: 0 loss: tensor(27.3412)
epoch: 24 batch: 0 loss: t

KeyboardInterrupt: 

In [ ]:
plt.plot(losses)

In [ ]:
losses[-1]

In [ ]:
plt.plot(losses)